In [1]:
import pdfplumber
import pandas as pd
import arabic_reshaper
from bidi.algorithm import get_display

In [2]:
all_data = [["Name", "Code", "Physique 1", "Moyenne_1", "Credit_1", "Algebre 1", "Analyse 1", "Moyenne_2", "Credit_2", "Algorithmique et Structure de Donné 1", "Structure Machine 1", "Moyenne_3", "Credit_3", "Langue Etranger 1", "Expression Scientifique Terminologie", "Moyenne_4", "Credit_4", "Semestre Credit", "Moyenne Semestre"]]

i = 0

def reshape_arabic(text):
    try:
        return get_display(arabic_reshaper.reshape(text)) if text else text
    except Exception:
        return text 

with pdfplumber.open("../data/deliberation.pdf") as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        table = page.extract_table()
        if table:
            table = table[2:]
            reshaped_table = [[reshape_arabic(cell) for cell in row] for row in table]
            all_data.extend(reshaped_table)


df = pd.DataFrame(all_data)
df.to_csv("../data/students_data.csv", index=False, quoting=1, sep=",", header=False)

print(f"✅ Extracted {len(df)} rows successfully!")


✅ Extracted 1441 rows successfully!
